# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
temp = df.copy() #Подсмотрел все это у прошлого проверяющего)))
list_c = ['calls', 'minutes', 'messages', 'mb_used', 'is_ultra']
print(temp.info())
for col_l in list_c:
  print('-'* 25)
  print(col_l, temp[col_l].sort_values().unique())
  print(col_l,': кол-во NaN',temp[col_l].isna().sum(),
        ', процент NaN', round(temp[col_l].isna().mean()*100,2),'%')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None
-------------------------
calls [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104.

**Вывод:**

Данные уже обработаны и готовые, так что можем приступить к обучению.



## Разбейте данные на выборки

Для корректного обучения разобьем дата сет на 3 выборки - обучающую, валидационную и тестовую.

Разобьем в пропорции 60%/20%/20%

In [5]:
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']

In [6]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.2, random_state=1234)

In [7]:
features_train, features_test, target_train, target_test = train_test_split(
    features_train, target_train, test_size=0.25, random_state=1234)

In [8]:
print(target.shape, target_train.shape, target_test.shape, target_test.shape)

(3214,) (1928,) (643,) (643,)


**Вывод:**
Разбили датасет на 3 выборки

* target_train - обучающая 60%
* target_test - тестовая 20%
* target_test - валидационная 20%

## Исследуйте модели

**DecisionTreeClassifier**

In [9]:
best_model = None
best_result = 0
best_depth = 0
for depth in range(1, 11):
    model_decision = DecisionTreeClassifier(random_state=1234, max_depth=depth)
    model_decision.fit(features_train, target_train)
    predictions = model_decision.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model_decision = model_decision
        best_result = result
        best_depth = depth
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result, "Глубина дерева:", best_depth)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.7884914463452566 Глубина дерева: 3


In [10]:
best_model = None
best_result = 0
best_depth = 0
best_min_samples_split = 0
best_min_samples_leaf = 0
for depth in range(1, 11):
    for split in range(2, 6):
        for leaf in range(1, 10):
            model_decision = DecisionTreeClassifier(random_state=1234, max_depth=depth, min_samples_split = split,
                                               min_samples_leaf = leaf)
            model_decision.fit(features_train, target_train)
            predictions = model_decision.predict(features_valid)
            result = accuracy_score(target_valid, predictions)
            if result > best_result:
                best_model_decision = model_decision
                best_result = result
                best_depth = depth
                best_min_samples_split = split
                best_min_samples_leaf = leaf
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result, "Глубина дерева:", best_depth,
      'Минимальное количество примеров для разделения', best_min_samples_split,
      'Минимальное количество объектов в листе', best_min_samples_leaf)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.7993779160186625 Глубина дерева: 10 Минимальное количество примеров для разделения 2 Минимальное количество объектов в листе 9


**RandomForestClassifier**

In [11]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        model_forest = RandomForestClassifier(random_state=1234, n_estimators=est, max_depth=depth)
        model_forest.fit(features_train, target_train)
        predictions_valid = model_forest.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model_forest = model_forest
            best_result = result
            best_est = est
            best_depth = depth
print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result,
      "Количество деревьев:", best_est, "Максимальная глубина:", depth)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.8149300155520995 Количество деревьев: 20 Максимальная глубина: 10


In [12]:
best_model = None
best_result = 0
best_est = 0
best_depth = 0
best_min_samples_split = 0
best_min_samples_leaf = 0
for est in range(10, 51, 10):
    for depth in range (1, 11):
        for split in range(2, 4):
            for leaf in range(1, 5):
                model_forest = RandomForestClassifier(random_state=1234, n_estimators=est, max_depth=depth,
                                                  min_samples_split=split, min_samples_leaf = leaf)
                model_forest.fit(features_train, target_train)
                predictions_valid = model_forest.predict(features_valid)
                result = accuracy_score(target_valid, predictions_valid)
                if result > best_result:
                    best_model_forest = model_forest
                    best_result = result
                    best_est = est
                    best_depth = depth
                    best_min_samples_split = split
                    best_min_samples_leaf = leaf
print("Accuracy наилучшей модели случайного леса на валидационной выборке:", best_result,
      "Количество деревьев:", best_est, "Максимальная глубина:", depth,
      'Минимальное количество примеров для разделения', best_min_samples_split,
     'Минимальное количество объектов в листе', best_min_samples_leaf)

Accuracy наилучшей модели случайного леса на валидационной выборке: 0.8242612752721618 Количество деревьев: 30 Максимальная глубина: 10 Минимальное количество примеров для разделения 2 Минимальное количество объектов в листе 4


**LogisticRegression**

In [13]:
model_logistic = LogisticRegression(random_state=1234, solver='lbfgs', max_iter=500)
model_logistic.fit(features_train, target_train)
predictions_valid = model_logistic.predict(features_valid)
result = accuracy_score(target_valid,predictions_valid)
print("Accuracy наилучшей модели на валидационной выборке:", result)

Accuracy наилучшей модели на валидационной выборке: 0.7325038880248833


In [14]:
best_model_logistic = None
best_result = 0
best_depth = 0
solvers = ['lbfgs', 'liblinear']
for depth in range(100, 1000, 100):
    for sol in solvers:
        model_logistic = LogisticRegression(random_state=1234, solver=sol, max_iter=depth)
        model_logistic.fit(features_train, target_train)
        predictions_valid = model_logistic.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model_logistic = model_logistic
            best_result = result
            best_depth = depth
            best_sol = sol

print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result, "Количество итераций:", best_depth,
      'Алгоритм, используемый для оптимизации', best_sol)

Accuracy лучшей модели дерева решений на валидационной выборке: 0.7325038880248833 Количество итераций: 100 Алгоритм, используемый для оптимизации lbfgs


**Вывод:**

Обучая три разныемодели были полученны лучшие метрики качества:

* Качество модели "Дерево предсказания": 0.78, Глубина дерева: 3
* Качество модели "Случайный лес": 0.81, Количество деревьев: 20, Максимальная глубина: 10
* Качество модели "Логистическая регрессия": 0.73

Лучше всего себя проявила модель "Случайный лес"

**Вывод:**

Обучая три разныемодели были полученны лучшие метрики качества:

* Accuracy лучшей модели "Дерево предсказания": 0.79
* Accuracy лучшей модели "Случайный лес": 0.82
* Accuracy лучшей модели "Логистическая регрессия": 0.73

Лучше всего себя проявила модель "Случайный лес"

## Проверьте модель на тестовой выборке

**DecisionTreeClassifier**

In [15]:
predictions_test1 = best_model_decision.predict(features_test)
result_test = accuracy_score(target_test,predictions_test1)
print("Accuracy модели дерева решений на тестовой выборке:", result_test)

Accuracy модели дерева решений на тестовой выборке: 0.80248833592535


**RandomForestClassifier**

In [16]:
predictions_test2 = best_model_forest.predict(features_test)
result_test = accuracy_score(target_test,predictions_test2)
print("Accuracy модели случайного леса на тестовой выборке:", result_test)

Accuracy модели случайного леса на тестовой выборке: 0.8055987558320373


**LogisticRegression**

In [17]:
predictions_test3 = model_logistic.predict(features_test)
result_test = accuracy_score(target_test,predictions_test3)
print("Accuracy модели случайного леса на тестовой выборке:", result_test)

Accuracy модели случайного леса на тестовой выборке: 0.7262830482115086


**Вывод:**

После проверки на тестовой выборке:

* Качество модели "Дерево предсказания": 0.79
* Качество модели "Случайный лес": 0.81
* Качество модели "Логистическая регрессия": 0.72

так же лучше всего себя проявила модель "Случайный лес"

## (бонус) Проверьте модели на адекватность

In [18]:
dummy = DummyClassifier(strategy="most_frequent", random_state=777)
dummy.fit(features_train, target_train)
dummy.score(features_test, target_test)

0.6998444790046656

**Вывод:**

Простейшая модель ошибается в 30% случаев

Выбранная модель ошибается в 19% случаев

**Модель адекватна.**

# Общий вывод:
1. Разбили датасет на 3 выборки: обучающую - 60%, валидационную - 20% и тестовую - 20%.
2. Исследовали 3 модели: дерева решений, случайного леса и логистической регрессии.
3. Лучше всего себя проявила модель "Случайный лес" как на валидационной (0,82), так и на тестовой выборке(0,81).
4. После проверки модели убедились в ее адекватности.